In [ ]:
import pyodbc
import pandas as pd

# Function to create a database connection
def create_connection(server, database):
    conn = None
    try:
        conn = pyodbc.connect("DRIVER={SQL Server};SERVER=" + server + ";DATABASE=" + database + ";Trusted_Connection=yes")
        print("Connected to SQL Server")
    except pyodbc.Error as e:
        print(e)
    return conn

# Function to create a dataframe out of csv file
def create_dataframe(filepath):
    file = pd.read_csv(filepath)
    data = pd.dataframe(file)
    return data



In [ ]:
# Function to create table in SQL server from dataframe
def create_table_with_inferred_schema(conn, data, table_name, pk_dtype, primary_key=None):
    columns = list(data.columns)
    cursor = conn.cursor()
    # Dictionary to map pandas data types to SQL Server data types
    type_mapping = {
        'object': 'VARCHAR(MAX)',
        'int64': 'INT',
        'float64': 'FLOAT',
        'datetime64': 'DATETIME'
        # Add more mappings as needed for other data types
    }
    # Infer data types for each column
    column_types = [type_mapping[str(data[col].dtype)] for col in columns]
    # Override the data type for the primary key column if specified
    if primary_key:
        column_types[columns.index(primary_key)] = pk_dtype  # Change the data type to INT for example
    # Construct the SQL query to create the table with inferred schema
    query = f"IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}')"
    query += f" CREATE TABLE {table_name} ({', '.join([f'{col} {data_type}' for col, data_type in zip(columns, column_types)])}"
    if primary_key:
        query += f", PRIMARY KEY ({primary_key})"
    query += ")"
    try:
        # Execute the SQL query
        cursor.execute(query)
        conn.commit()
        print(f"Table '{table_name}' created successfully with inferred schema.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
# Iterate over each row in the DataFrame
def insert_row_into_database(dataframe, table_name, conn):
    cursor = conn.cursor()
    # Get the primary key column name from the database
    cursor.execute(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE WHERE OBJECTPROPERTY(OBJECT_ID(CONSTRAINT_SCHEMA + '.' + QUOTENAME(CONSTRAINT_NAME)), 'IsPrimaryKey') = 1 AND TABLE_NAME = '{table_name}'")
    primary_key_column = cursor.fetchone()[0]

    # Iterate over each row in the DataFrame
    for index, row in dataframe.iterrows():
        try:
            # Check if the row already exists in the database using the primary key column
            cursor.execute(f"SELECT COUNT(*) FROM {table_name} WHERE {primary_key_column} = ?", (row[primary_key_column],))
                
            if cursor.fetchone()[0] == 0:
                # If the row doesn't exist, insert it into the database
                columns = ', '.join(dataframe.columns)
                placeholders = ', '.join(['?' for _ in range(len(dataframe.columns))])
                sql_query = f'''
                    INSERT INTO {table_name} ({columns}) 
                    VALUES ({placeholders})
                '''
                values = tuple(row[col] for col in dataframe.columns)
                cursor.execute(sql_query, values)
                print(f"Inserted row {index}")
                conn.commit()  # Commit the changes after each successful insertion
                break
            else:
                print(f"Row {index} already exists in the database")
                    
        except (pyodbc.Error, ValueError) as e:
            print(f"Error inserting row {index}: {e}")
            continue

In [ ]:
# Main function
import schedule
import time

def main():
    server = "server_name"
    database = "database_name"
    table_name = "your_table"
    file_path = "your_path"
    
    conn = create_connection(server, database)
    create_dataframe(filepath)
    create_table_with_inferred_schema(conn, data, table_name, pk_dtype, primary_key=None)
    
    # Counter to track the number of iterations
    iteration_count = 0

    def my_function():
        global iteration_count
        # Call the insert_row_into_database function
        insert_row_into_database(dataframe, table_name, conn)
        iteration_count += 1
        if iteration_count >= 10:  # Stop after 5 iterations
            schedule.clear()  # Clear all scheduled jobs
            print("Schedule stopped.")
            %killbgscripts  # Terminate any background scripts
            
    # Schedule the function to run every 1 minutes
    schedule.every(1).minutes.do(my_function)

    # Run the scheduler loop
    while True:
        schedule.run_pending()
        time.sleep(1)  # Sleep for 1 second to avoid high CPU usage
    
if _name_ == "_main_":
    main()